In [ ]:
import pandas as pd
import numpy as np

import pymc as pm
import arviz as az
import seaborn as sns

In [2]:
# true values
p_true = np.array([0.10, 0.095, 0.105])  # index 0 is control; others are treatment variants

# sample size
n_total = 10000

weights_true = np.array([0.5, 0.3, 0.2])  # weights for control and treatment variants
group_labels = ['control'] + [f'treatment_{i}' for i in range(1, len(weights_true))]

n_samples = (weights_true * n_total).astype(int)

# Initialize random number generator
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

In [ ]:
# user_id, group, converted

def generate_dataframe(p, n, group_name):
    return pd.DataFrame({
        'user_id': np.arange(n),
        'group': [group_name for _ in range(n)],
        'converted': np.random.binomial(1, p, n)
    })

data_lst = []

for i, (p, n, group_name) in enumerate(zip(p_true, n_samples, group_labels)):
    data_lst.append(generate_dataframe(p, n, group_name))

data = pd.concat(data_lst)
data

In [ ]:
sns.countplot(x='converted', hue='group', data=data)

In [5]:
# pymc simulation of the difference in means between groups
# priors: bernoilli distribution
# likelihood: from data
# posterior: beta
basic_model = pm.Model()

with basic_model:
    # Priors for unknown model parameters (use vectors for multiple groups)
    p = pm.Beta('p', alpha=1, beta=1, shape=len(group_labels))
    
    # Likelihood (sampling distribution) of observations    
    for k, group_label in enumerate(group_labels):
        pm.Bernoulli(f'obs_{k}', p=p[k], observed=data[data['group'] == group_label]['converted'])
            
        # p[k] best than p[i] for any k!=i, that is, p[k] - max(p_without_k)
        p_without_k_max = pm.math.max(p[[i for i in range(len(group_labels)) if i != k]])
        pm.Deterministic(f'delta_{k}', p[k] - p_without_k_max)

    # Define the difference between each group's conversion rate and a reference
    # Here, we use the maximum conversion rate among all groups as the reference
    p_max = pm.math.max(p)
    
    # Loss function for each group
    # The loss is the opportunity loss when choosing group k over the best group
    loss = pm.Deterministic('loss', -(p - p_max))
    # loss[k]: expected loss if group k is rolled out


In [ ]:
# requires brew/choco install graphviz in your local machine and add python package to pyproject.toml as well
pm.model_to_graphviz(basic_model)

In [ ]:
n_draws = 1000
n_chains = 4
n_tune = 1000

with basic_model:
    # Sample from the posterior
    trace = pm.sample(
        draws=n_draws,
        chains=n_chains,
        tune=n_tune,
        random_seed=RANDOM_SEED,
        progressbar=True
    )

In [ ]:
trace

In [ ]:
az.plot_trace(trace, combined=True)

In [ ]:
# Analyze the posterior distributions

# Summary statistics
print(az.summary(trace))

# Plot posterior distributions
az.plot_posterior(trace)

In [29]:
# Extract posterior samples
delta_samples = np.stack([trace.posterior[f'delta_{k}'].values for k in range(len(group_labels))], axis=-1)
loss = trace.posterior['loss'].values

In [ ]:
delta_samples.shape, loss.shape

In [ ]:
# Probability that treatment is better than control
prob_better_treatment = np.mean(delta_samples > 0, axis=(0, 1))
print(f"Probability that treatment is better than the rest: {prob_better_treatment}")

# Expected loss if control is rolled out (missed opportunity)
expected_loss_per_group = loss.mean(axis=(0, 1))
print(f"Expected loss if variant is rolled out: {expected_loss_per_group}")


loss_ratio = expected_loss_per_group / expected_loss_per_group[0]
print(f"Ratio of expected losses: {loss_ratio}")

In [ ]:
az.plot_energy(trace)

In [ ]:
az.plot_forest(trace, combined=True, hdi_prob=0.95, r_hat=True)